In [ ]:

output_df = contig_to_chr.sort_values(by=['chr', 'min_chr_map_start'])


In [2]:
import numpy as np
import pandas as pd
from Bio import SeqIO
import networkx as nx

In [3]:
mappings_df = pd.read_csv('/mnt/c/users/tkoti/Desktop/Genomes/gap_fill_automation/C_Nigoni/assembly.fasta_mapped_to_caenorhabditis_nigoni.PRJNA384657.WBPS19.genomic.fa.sorted.paf', delimiter = '\t', header = None)
mappings_df.columns = (
['contig', 'contig_length', 'contig_map_start', 'contig_map_end', 
'strand', 'chr', 'chr_length', 'chr_map_start', 'chr_map_end', 'matched_bases', 'matched_total', 'mapping_quality'] + 
[f"sup_{i}" for i in range(mappings_df.shape[1] - 12)]
)

In [7]:
def get_readlength_stats(reads_file):
    read_lengths = (len(record.seq) for record in SeqIO.parse(reads_file, "fasta"))
    return pd.Series(read_lengths).describe()

In [8]:
read_stats = get_readlength_stats('/mnt/c/users/tkoti/Desktop/Genomes/raw_reads/C_nigoni_JU1422.fasta')

KeyboardInterrupt: 

In [ ]:
organelle_options = ['MT', 'mt', 'mitochondrion', 'chloroplast', 'plastid']  # Add more organelle identifiers as needed

organelle_contigs = mappings_df[mappings_df['chr'].isin(organelle_options)] # For now, ok, in future, filter out chloroplasts and use regex to account for multiple formats

mappings_df = mappings_df[(~mappings_df['chr'].isin(organelle_options)) & (mappings_df['contig_length'] > min_contig_length) & (mappings_df['sup_0'] == 'tp:A:P')]

In [ ]:
prom_map = {item : {chr : 0 for chr in mappings_df['chr'].unique()} for item in mappings_df['contig'].unique()}
for key, value in prom_map.items():
    for chr, count in value.items():
        prom_map[key][chr] += int(mappings_df[(mappings_df['contig'] == key) & (mappings_df['chr'] == chr)]['matched_total'].sum())


contig_to_chr = pd.DataFrame(sorted([(item, next(key for key,value in prom_map[item].items() 
                                if value == max(prom_map[item].values()))) for item in prom_map.keys()], 
                                key = lambda x: x[1]), columns=['contig', 'chr'])

In [ ]:
strand_modes, max_chr_map_ends, min_chr_map_starts = [], [], [] #latest edit
strand_map = {'+': 1, '-': -1}  # Map strand to numerical values for mode calculation
for item in contig_to_chr.itertuples(index=False):

    poi_df = mappings_df[(mappings_df['contig'] == item[0]) & (mappings_df['chr'] == item[1])]

    ############# Calculate strand mode using a weighted approach

    matched_total = poi_df['matched_total'].sum()
    length_ratio_series = (poi_df['contig_map_end'] - poi_df['contig_map_start']) / matched_total
    criterion = (poi_df['strand'].apply(lambda x: strand_map[x] if x in strand_map else np.nan) @ length_ratio_series)
    if criterion >= 0:
        strand_modes.append('+')
    else:
        strand_modes.append('-')

    #mode_series = poi_df['strand'].mode() <- old method, but does not work well for complex regions with multiple mappings
    #############

    

    #max_end = mappings_df[(mappings_df['contig'] == item[0]) & (mappings_df['chr'] == item[1]) & (mappings_df['mapping_quality'] == 60)]['chr_map_end'].max()
    max_end = poi_df[poi_df['mapping_quality'] >= phred_threshold]['chr_map_end'].max() #latest edit
    #min_start = mappings_df[(mappings_df['contig'] == item[0]) & (mappings_df['chr'] == item[1]) & (mappings_df['mapping_quality'] == 60)]['chr_map_start'].min() #latest edit

    min_start = poi_df[poi_df['mapping_quality'] >= phred_threshold]['chr_map_start'].min() #latest edit
    #This will also have some bugs, check when more data available
    if pd.isnull(max_end):
        print(f"Warning: No valid chr_map_end found for contig {item[0]} on chr {item[1]}.")
    max_chr_map_ends.append(max_end if pd.notnull(max_end) else None)
    min_chr_map_starts.append(min_start if pd.notnull(min_start) else None) #latest edit


contig_to_chr['strand_mode'] = strand_modes
contig_to_chr['min_chr_map_start'] = min_chr_map_starts  # latest edit
contig_to_chr['max_chr_map_end'] = max_chr_map_ends


####################################################

output_df = contig_to_chr.sort_values(by=['chr', 'min_chr_map_start'])